# AI-Assisted Data Curation Toolkit

This notebook demonstrates the AI-Assisted Data Curation Toolkit. It is capable of suggesting harmonizations from a source data model into a target data model using AI-backed approaches, but leaving the expert curator in complete control.

## Setup

In [ ]:
# If you are actively working on related *.py files and would like changes to reload automatically into this notebook
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import json

from harmonization.interactive import (
    get_interactive_table_for_suggestions,
    get_nodes_and_properties_df,
)
from harmonization.simple_data_model import (
    SimpleDataModel,
    get_data_model_as_node_prop_type_descriptions,
)
from harmonization.harmonization_approaches.similarity_inmem import (
    SimilaritySearchInMemoryVectorDb,
)
from harmonization.harmonization_approaches.rag import (
    RetrievalAugmentedGeneration,
)
from harmonization.harmonization_approaches.embeddings import (
    MedGemmaEmbeddings,
    QwenEmbeddings,
)

Set available GPUs (skip this step is using CPUs)

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"  # change as necessary

## Use a Harmonization Approach to get Suggestions

### Get Input Data

- A `source data model` you want to harmonize from
- A `target data model` you want to harmonize to

For this initial example, you can just using hard-coded examples.

- The `example_synthetic_source_model.json` is a synthetically generated model for example purposes
- The `example_real_source_model.json` is a real original study before ingestion into the NHLBI BioData Catalyst ecosystem (e.g. not yet harmonized)
- The `target data model` example is the **NHLBI BioData Catalyst Gen3 Data Dictionary v4.6.5** (latest version as of 21 AUG 2025)

You can change this to supply your own source model, so long as the format follows the example. Similarly for target model. The source model will eventually come from a connection to a previously released AI-backed tool for Schema Generation, allowing this entire flow to start from arbitrary TSVs.

In [ ]:
# source_file = "./examples/example_synthetic_source_model.json"
source_file = "./examples/example_real_source_model.json"

target_file = "./examples/example_target_model_BDC.json"

with open(source_file, "r") as f:
    input_source_model = json.load(f)

input_source_model = SimpleDataModel.get_from_unknown_json_format(
    json.dumps(input_source_model)
)

with open(target_file, "r") as f:
    input_target_model = json.load(f)

input_target_model = SimpleDataModel.get_from_unknown_json_format(
    json.dumps(input_target_model)
)

In [ ]:
print("Source Model")
input_source_model.get_property_df()

In [ ]:
print("Target Model")
input_target_model.get_property_df()

### Use a Specific Harmonization Approach to get Suggestions

In [ ]:
# Uncomment desired embedding model
# embedding_fn = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
# embedding_fn = MedGemmaEmbeddings(model_name="google/medgemma-4b-it")
# embedding_fn = MedGemmaEmbeddings(model_name="google/embeddinggemma-300m")
embedding_fn = QwenEmbeddings(model_name="Qwen/Qwen3-Embedding-0.6B")

batch_size = 32


# Similarity Search Approach
# :62 b/c of limitation on chromadb collection names
harmonization_approach = SimilaritySearchInMemoryVectorDb(
    vectordb_persist_directory_name=f"{os.path.basename(target_file)[:53]}-{embedding_fn.model.name_or_path.split("/")[-1][:5]}-0",
    input_target_model=input_target_model,
    embedding_function=embedding_fn,
    batch_size=batch_size,
)

max_suggestions_per_property = 10
# set threshold low to just get top properties no matter what
score_threshold = 0

suggestions = harmonization_approach.get_harmonization_suggestions(
    input_source_model=input_source_model,
    input_target_model=input_target_model,
    score_threshold=score_threshold,
    k=max_suggestions_per_property,
)
"""

# RAG Approach

llm_model_name = "meta-llama/Llama-3.1-8B-Instruct"
# set how many suggestions from similarity search as the choices for llm, update llm_max_model_len accordingly.
similarity_search_k_value = 30
# set threshold low to just get top properties no matter what
similarity_search_score_threshold = 0.1
# if the prompt is too long, update max_model_len accordingly
llm_max_model_len = 8192
# the approach searches more candidates, but only output max number of suggestions
max_suggestions_num = 5
# the embedding occupys memory, lower llm's gpu_memory_utilization if llm runs into memory issue. the Default is 0.9.
llm_gpu_memory_utilization = 0.7

# :62 b/c of limitation on chromadb collection names
harmonization_approach = RetrievalAugmentedGeneration(
    vectordb_persist_directory_name=f"{os.path.basename(target_file)[:53]}-{embedding_fn.model.name_or_path.split("/")[-1][:5]}-0",
    input_target_model=input_target_model,
    embedding_function=embedding_fn,
    embedding_batch_size=batch_size,
    similarity_search_k_value=similarity_search_k_value,
    similarity_search_score_threshold=similarity_search_score_threshold,
    llm_gpu_memory_utilization=llm_gpu_memory_utilization,
    max_suggestions_num=max_suggestions_num,
    llm_max_model_len=llm_max_model_len,
    llm_model_name=llm_model_name,
)

suggestions = harmonization_approach.get_harmonization_suggestions(
    input_source_model=input_source_model,
    input_target_model=input_target_model,
)
"""

### Visualize Suggestions

In [ ]:
table_df = suggestions.to_simlified_dataframe()
table_df.sort_values(by="Similarity", ascending=False, inplace=True)
table_df

In [ ]:
# Group by 'Original Node.Property' and find the index of max similarity for each group
idx = table_df.groupby("Original Node.Property")["Similarity"].idxmax()

# Filter DataFrame using the indices found above
filtered_df = table_df.loc[idx]
filtered_df.drop(columns=["Original Description", "Target Description"], inplace=True)
filtered_df.sort_values(by="Similarity", ascending=False, inplace=True)
filtered_df

### Create Interactive Table for Selecting Suggestions

In [ ]:
table = get_interactive_table_for_suggestions(
    table_df,
    column_for_filtering=1,
    # additional config for the interactive table
    maxBytes="2MB",
    pageLength=50,
)
table

> **Don't see the table or see an error above?** Try restarting the kernel, then try restarting jupyter lab (if that's what you're using). The installs for AnyWidgets might not be picked up yet.

> **Dark Theme?** If you're using a dark theme, you might need to switch to light for the table to display properly. 

> **Using VS Code Jupyter Extension?** Links might not work

To use the selections above, record them below in `manual_selection_indexes` or ue multi-select in the above table and the below will automatically use those. 

In [ ]:
# Fill this out manually as you go, or we'll use the table selections
manual_selection_indexes = []  # [1, 8, 24, ...]

selected_rows = manual_selection_indexes or table.selected_rows

print(f"Selected Suggestions: {selected_rows}")

In [ ]:
table_df.iloc[selected_rows]

In [ ]:
table_df.iloc[selected_rows].to_csv(
    "./selected_suggestions.tsv",
    index=False,
    na_rep="N/A",
    sep="\t",
    quotechar='"',
)